<a href="https://colab.research.google.com/github/choarauc/form_ch/blob/main/geocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyairtable

In [ ]:
from pyairtable import Table
import os
import pandas as pd

In [ ]:
AIRTABLE_API_KEY = "keyE4yOUAuBjB6jWW"
AIRTABLE_BASE_ID = "app85m0W3SjLXqz0R"
AIRTABLE_TABLE_NAME = "Etudiant"
table = Table(AIRTABLE_API_KEY, AIRTABLE_BASE_ID, AIRTABLE_TABLE_NAME)

In [ ]:
rows = table.all(max_records=1000)
rows[0]

{'id': 'rec07iXfwMcCeBpza',
 'createdTime': '2023-06-09T09:17:27.000Z',
 'fields': {'Code postal': 31290,
  'Prenom': 'Jacques',
  'Ville': 'Villefranche de Lauragais'}}

In [ ]:
df = pd.DataFrame([row["fields"] for row in rows])

In [ ]:
df

In [ ]:
#@title
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy.extra.rate_limiter import RateLimiter

# Géocodage des villes avec geopy
geolocator = Nominatim(user_agent='my_geocoder')
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df['Location'] = df['Ville'].apply(geocode)

# Extraction des coordonnées géographiques (latitude, longitude)
df['Latitude'] = df['Location'].apply(lambda x: x.latitude if x else None)
df['Longitude'] = df['Location'].apply(lambda x: x.longitude if x else None)

# Filtrage des villes géocodées avec des coordonnées valides
geocoded = df.dropna(subset=['Latitude', 'Longitude'])

# Calcul de l'épicentre en utilisant la moyenne des coordonnées
epicentre_latitude = geocoded['Latitude'].mean()
epicentre_longitude = geocoded['Longitude'].mean()

# Création de la carte
m = folium.Map(location=[epicentre_latitude, epicentre_longitude], zoom_start=5)

# Ajout des points de villes avec prénoms et distance
for _, row in geocoded.iterrows():
    ville_coords = (row['Latitude'], row['Longitude'])
    distance = geodesic(ville_coords, (epicentre_latitude, epicentre_longitude)).kilometers
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        tooltip=row['Ville'],
        icon=folium.Icon(color='blue'),
        popup=f"Prénom: {row['Prenom']}<br>Distance du point de rdv: {distance:.2f} km"
    ).add_to(m)

# Ajout de l'épicentre 
folium.Marker(
    location=[epicentre_latitude, epicentre_longitude],
    tooltip='Le sud ouest c est pas ici :) ',
    icon=folium.Icon(color='red')
).add_to(m)

# Calcul des itinéraires à vol d'oiseau sur la carte
for _, row in geocoded.iterrows():
    start_coords = (row['Latitude'], row['Longitude'])
    end_coords = (epicentre_latitude, epicentre_longitude)
    folium.PolyLine(
        locations=[start_coords, end_coords],
        color='gray',
        weight=2
    ).add_to(m)

# Affichage 
m